# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [18]:
from models.arcs import generate, GenPhiloTextA
from utilities.loaders import load_lookup_table, load_hyper_params
from utilities.preprocessors import map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load saved hyper parameters and lookup table

In [19]:
chars = load_lookup_table('./saved/misc/char_to_idx')
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [20]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Load same hyper params used in training model

In [21]:
hyper_params = load_hyper_params('./saved/misc/hyper_params.json')
hyper_params

{'d': 'notes',
 'model': 'GenPhiloTextA',
 'emb_dim': 256,
 'n_a': 512,
 'T_x': 100,
 'dense_layers_dims': [],
 'batch_size': 128,
 'alpha': 0.001,
 'lambda_': 0.8,
 'drop_prob': 0.4,
 'n_epochs': 100,
 'normalize': False,
 'n_unique': 57}

# Declare sample input to build inference model and access .summary()

In [22]:
sample_input = tf.random.uniform(shape=(1, hyper_params['T_x']), minval=0, maxval=hyper_params['n_unique'] - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, hyper_params['n_a']))
sample_c = tf.zeros(shape=(1, hyper_params['n_a']))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [23]:
saved_model = GenPhiloTextA(
    emb_dim=hyper_params['emb_dim'], 
    n_a=hyper_params['n_a'],
    n_unique=hyper_params['n_unique'],
    dense_layers_dims=hyper_params['dense_layers_dims'] + [hyper_params['n_unique']],
    drop_prob=hyper_params['drop_prob'],
    normalize=hyper_params['normalize'])
saved_model(sample_input)

<tf.Tensor: shape=(1, 100, 57), dtype=float32, numpy=
array([[[-0.00782075,  0.00528927, -0.00162494, ..., -0.00052786,
         -0.00558462,  0.00939467],
        [-0.01102713,  0.00778486, -0.00142314, ...,  0.0012905 ,
          0.00047561,  0.00640498],
        [-0.01599064,  0.00845143,  0.00405058, ...,  0.00092203,
         -0.00359961,  0.00380738],
        ...,
        [-0.00086698,  0.00616083, -0.00522919, ...,  0.00698488,
         -0.00014025,  0.00950269],
        [ 0.00201925,  0.00132826,  0.00163919, ...,  0.01088386,
         -0.00627568,  0.00946886],
        [ 0.00230826, -0.00048635, -0.00980438, ...,  0.00404211,
          0.00120269,  0.00975345]]], dtype=float32)>

In [24]:
sample_input = tf.random.uniform(shape=(1, hyper_params['T_x'] + 10), minval=0, maxval=hyper_params['n_unique'] - 1, dtype=tf.int32)
saved_model(sample_input)

<tf.Tensor: shape=(1, 110, 57), dtype=float32, numpy=
array([[[-6.5240534e-03, -6.7024650e-03,  3.2486082e-03, ...,
         -6.9395639e-05,  1.0831628e-03, -1.1644097e-02],
        [-1.6318024e-03,  3.0069391e-03, -2.2086606e-03, ...,
         -5.4106326e-04,  1.6853565e-03, -2.5335965e-03],
        [-3.7682438e-03, -4.6216222e-03,  1.2948255e-03, ...,
          1.1441605e-03,  5.5005923e-03,  5.1969755e-04],
        ...,
        [-2.1647583e-03,  5.9350487e-03, -1.8212622e-02, ...,
         -1.1829675e-02, -3.1432847e-03, -1.2551094e-03],
        [-1.0837550e-02,  6.9573564e-03, -1.4214615e-02, ...,
         -8.3052041e-03, -8.6048050e-03,  5.8352472e-03],
        [-1.1567408e-02,  1.1693357e-02, -1.7794449e-02, ...,
         -4.5692697e-03, -2.1635119e-03,  3.9152326e-03]]], dtype=float32)>

In [25]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_1/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[-0.03607858, -0.04805228, -0.02559731, ..., -0.00985156,
          0.00506531,  0.00093282],
        [ 0.02584629,  0.04303923, -0.0090935 , ...,  0.0067636 ,
         -0.03762203, -0.03750806],
        [-0.03190702,  0.0374064 , -0.00448339, ..., -0.02482504,
         -0.02452344,  0.03351477],
        ...,
        [-0.00190307, -0.04691879, -0.02637919, ..., -0.02494066,
         -0.0233341 , -0.03935435],
        [-0.04644127, -0.01752782,  0.0431559 , ..., -0.01798801,
         -0.04167613,  0.00545577],
        [ 0.04264149,  0.0370441 , -0.03004944, ...,  0.0444735 ,
         -0.04037479, -0.01980265]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_1/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[-0.02449795,  0.01973499,  0.03153598, ..., -0.01856035,
         -0.00251354,  0.01690923],
        [-0.03047572,  0.00180775,  0.03670113,

In [26]:
saved_model.summary()

Model: "gen_philo_text_a_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 character-lookup (Embeddin  multiple                  14592     
 g)                                                              
                                                                 
 lstm-layer (LSTM)           multiple                  1574912   
                                                                 
 dense-layer-0 (Dense)       multiple                  29241     
                                                                 
Total params: 1618745 (6.18 MB)
Trainable params: 1618745 (6.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_a_100_3.0299.h5')

In [28]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_1/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[-2.27964771e-31, -1.04065716e-31,  6.76143058e-32, ...,
          5.78990514e-33, -6.32452695e-35,  1.94203334e-32],
        [-2.33196533e-06, -5.57359442e-07,  7.22897767e-07, ...,
          3.29896963e-08,  4.24313072e-07, -7.51945777e-08],
        [-6.22333800e-06, -8.55494363e-06,  1.20172226e-05, ...,
          3.50521004e-05,  2.60806992e-05, -1.78583414e-05],
        ...,
        [ 1.16941873e-11,  5.07066784e-11,  2.42132052e-12, ...,
          1.95506927e-11, -9.94128564e-12,  5.03835099e-11],
        [-1.29197097e-09, -5.15809404e-08,  6.50408394e-09, ...,
          1.08325215e-08,  1.27126455e-07, -2.77360197e-08],
        [ 6.44838792e-07, -1.91505521e-07,  9.48950856e-08, ...,
          1.13875785e-06,  8.59064301e-07, -3.50037581e-08]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_1/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 arra

# generate characters using prompt

In [29]:
prompts = ["hello"]
pred_ids_a = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=1.0)
pred_ids_b = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.8)
pred_ids_c = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.6)
pred_ids_d = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.4)

tf.Tensor([[32 29 36 36 39]], shape=(1, 5), dtype=int64)
iteration 0
predicted logits: [[       -inf  0.03451424  3.0993068  -1.4558783  -1.2545254  -1.4891926
  -1.48162    -1.470978   -1.4589243   0.06746344 -1.4621998  -0.3321303
  -1.4430645  -1.4734247  -1.4767125  -1.4925032  -1.4616551  -1.4977362
  -1.491077   -1.4923937  -1.4801264  -1.5095773  -1.4366975  -1.3917115
  -1.3034109   2.1068554   0.4055695   0.9075593   1.1696806   2.479794
   0.8948436   0.63771284  1.6354678   2.0451353  -1.291325   -0.469146
   1.1744978   0.7348629   2.00982     2.0166864   0.33140537 -1.367948
   1.5217376   1.7782404   2.1739714   0.87940145  0.3724299   0.7176461
  -1.2924658   0.69691676 -1.3568192  -1.4502157  -1.4881953  -1.4739432
  -1.5011342  -1.3165642  -1.3051584 ]]
iteration 10
predicted logits: [[       -inf  0.0348281   3.108213   -1.4578725  -1.2558699  -1.4911515
  -1.4834833  -1.4728684  -1.4608581   0.06775467 -1.4640263  -0.33211765
  -1.4449506  -1.4753914  -1.4786851  -1.

In [30]:
pred_ids_a

<tf.Tensor: shape=(1, 255), dtype=int64, numpy=
array([[32, 29, 36, 36, 39,  2, 46, 44, 38, 45, 42,  2,  2, 33, 25, 39,
        32, 16, 45, 33, 44, 29, 28, 31, 32, 43, 38, 43, 29,  2, 29,  2,
        25, 20,  2, 38, 29, 29, 32, 33, 27, 45, 33, 43, 33,  2, 33, 44,
        29,  2, 54, 39, 37, 13, 30, 28,  2, 38, 38, 30,  2, 38, 45, 18,
         2, 44, 25, 37, 44, 29,  2, 33, 30, 29, 37,  9, 38, 46, 30, 42,
        37,  2, 45, 27, 49, 27, 46,  2, 29, 29,  2,  8, 22,  2, 36, 43,
        27, 44, 35, 42, 44,  2,  2, 40, 37, 32,  9,  2, 38, 28, 25, 32,
        42,  1, 42,  2, 42, 33, 49, 38, 24,  2, 44, 33,  2, 39, 39, 29,
         2,  2, 33,  2, 29,  2, 43, 38, 33, 38, 25, 39, 32, 29, 28, 38,
        29, 32, 33, 31, 38, 33, 29, 33,  2, 31,  4, 28, 14, 29, 49, 40,
        33, 28,  2, 38, 33, 39, 41, 31, 29, 25,  2, 33, 33, 42, 30, 29,
         1, 42, 42, 49, 42, 38, 36, 31, 45, 38, 44, 33, 31,  2, 43, 32,
        43, 40, 31, 36,  2, 27, 44, 32,  2,  2, 51, 45, 28, 25,  2, 42,
        56, 36, 

# Extract trainable layers

In [31]:
decode_predictions(pred_ids_a, idx_to_char)

'hello vtnur  iaoh3uitedghsnse e a8 neehicuisi ite ïom0fd nnf nu6 tamte ifem,nvfrm ucycv ee ): lsctkrt  pmh, ndahr\nr riyn? ti ooe  i e sninaohednehigniei g"d1eypid nioqgea iirfe\nrryrnlguntig shspgl cth  ´uda r…lo6l gc  eed   anortai  ehttii wa xw cre nerwo'

In [32]:
decode_predictions(pred_ids_b, idx_to_char)

"helloesoiit eae a ,l l   tnaa c tibmiuuweoe  n cebye iug : \n  oe ncew'oy   ama rsnt\n tn   fnyem or o hd&e j ee oewy  sahnoteir haan hf  wsm nsanco dsifil davlf dn eadfap msn,hwhan sesolhnoeatn hoiatnedi lti  iorie non b    n dtw te c ff a teaynets ong ae "

In [33]:
decode_predictions(pred_ids_c, idx_to_char)

'hellootb n   tet \nesia tr o   oeat  on i  s e er nooote nstorn t a re i aeteeoto  o mea ia oeeete  eie ntiioe       weoee   e  t  stosenne ae h e c t o he  e    a;ton pes awai ot  t et    tti cy ou     aaaia    e h ivtkcimn sohatu utta eassa  ea t ei n  e'

In [34]:
decode_predictions(pred_ids_d, idx_to_char)

'helloaon e     dt       ii   ar   io    e              i aeadieet n    h     ee  fe  t  an t   te n en h    en  t         aa u a   t    ndn  seo  eson    eta        t t  orei  t      a i  e  an  a  e nhee  na  tat et l e  t         s   eo nii     e      e'